# Showcase 2D KP

### Load Model & Dataset

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import src.real_bug_dataset
from tqdm.notebook import tqdm
from src.models.pose_resnet import BasicBlock, PoseResNet, Bottleneck
import src.bug_dataset
from src.real_bug_dataset import RealBugDataset
from src.eval.loss import JointsMSELoss
from src.eval.accuracies import accuracy
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
file_path = '../data/single_sungaya/label_names.txt'

with open(file_path) as file:
    label_names = file.readlines()
    label_names = [item.rstrip() for item in label_names]

In [ ]:
# Sungaya Synth
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))
reduceKeypoints = True
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir, reduced=reduceKeypoints, transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))
                 

In [ ]:
train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)
_,_ ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], generator=torch.Generator().manual_seed(42))

batch_size = 128
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))   

# Create Model
num_layers = 18
resnet_spec = {18: (BasicBlock, [2, 2, 2, 2]),
               34: (BasicBlock, [3, 4, 6, 3]),
               50: (Bottleneck, [3, 4, 6, 3]),
               101: (Bottleneck, [3, 4, 23, 3]),
               152: (Bottleneck, [3, 8, 36, 3])}
if reduceKeypoints:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 28)

else:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 62)


criterion = JointsMSELoss(use_target_weight=True)
model = model.to(device)
path = "../models/2d_horiz_vertical_flip/simple_2d_model_20220321-195443_128"
model.load_state_dict(torch.load(path))
model.eval()
trained_model = model

In [ ]:
# trained_model = liftingtrainer.model
trained_model.eval()
# Code get the first batch of results
with torch.no_grad():
    for data in test_dataloader:
        image = data['image']
        target = data['heatmap']
        file_name = data['file_name']
        kp = data['key_points_2D']
        visib = data['visibility']
        input_var = image.to(device, dtype=torch.float)
        heatmap_var = target.to(device, dtype=torch.float)

        predi = trained_model(input_var)
        break


In [ ]:
from src.eval.accuracies import get_max_preds
# Display a sample

sample = 7
limb = 10

name = file_name[sample]

img = image[sample].numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

pred = predi.cpu().numpy()
kps,_ = get_max_preds(pred)
acc = target.cpu().numpy()
acc_kp = ((kp[sample].numpy()*sungaya_dataset.std_2d)+sungaya_dataset.means_2d).astype(int)

vis = visib[sample].cpu().numpy()
print("Limb Visible", vis[limb] , acc_kp[limb])
fig, axes = plt.subplots(figsize=(8,6))
axes.imshow(img)

axes.scatter(acc_kp[limb,0], acc_kp[limb,1], marker='o', s=5,  color='b')
Drawing_uncolored_circle =plt.Circle( acc_kp[limb],
                                      5 ,
                                      fill = False )
plt.imshow(cv2.resize(pred[sample,limb], (256,256)), alpha = .4,cmap='hot')
axes.add_artist( Drawing_uncolored_circle )
# plt.colorbar()
plt.title(name)
plt.show()